In [1]:
import os
import re
import nltk
import pandas as pd
import numpy as np

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

regex resources: https://regexr.com/ and 
https://docs.python.org/3/library/re.html#

In [3]:
cd

/Users/andrew.vangilder


In [4]:
os.chdir('Desktop/Notebooks')

In [6]:
scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials/sheets-api-test-12bfaf791b73.json', scope)
gc = gspread.authorize(credentials)

In [7]:
spreadsheet_key = '1IfqJFR8yE8lcF-DJQ980UmscoxaZ9Rv1D8AdF4mwODU'

In [8]:
book = gc.open_by_key(spreadsheet_key)

In [9]:
worksheet = book.sheet1
crosswalk = worksheet.get_all_values()

In [10]:
spreadsheet_key = '1-SX2_WtGv-tORP-1PCXy29jacYxF7Sf572pHIiYcQeg'

In [11]:
book = gc.open_by_key(spreadsheet_key)

In [25]:
worksheet = book.sheet1
icd03 = worksheet.get_all_values()

In [ ]:
dataframe

In [27]:
def gsheet2df(gsheet):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!
    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.
    """
    header = gsheet[0]   # Assumes first line is header!
    values = gsheet[1:]  # Everything else is data.
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df


In [30]:
icd03_df = gsheet2df(icd03)

In [31]:
crosswalk_df = gsheet2df(crosswalk)

In [33]:
head1 = icd03_df.head()
head2 = crosswalk_df.head()

In [39]:
head1

,Site Recode,Site Description,Histology,Histology Description,Histology Behavior,Histology Behavior/Description
0,"C000-C006,C008-C009",LIP,800,NEOPLASM,8000/3,"Neoplasm, malignant"
1,"C000-C006,C008-C009",LIP,800,NEOPLASM,8001/3,"Tumor cells, malignant"
2,"C000-C006,C008-C009",LIP,800,NEOPLASM,8002/3,"Malignant tumor, small cell type"
3,"C000-C006,C008-C009",LIP,800,NEOPLASM,8003/3,"Malignant tumor, giant cell type"
4,"C000-C006,C008-C009",LIP,800,NEOPLASM,8004/3,"Malignant tumor, spindle cell type"


In [37]:
head2

,ICDO3,ICD10,ICDO3 Display,SNOMEDCT,SNOMEDCT Display
0,8850/3,C18.3,Liposarcoma of hepatic flexure of colon,48338005,Structure of right colic flexure
1,9727/3,C25.8,Precursor cell lymphoblastic lymphoma of overl...,372003004,Primary malignant neoplasm of pancreas
2,9738/3,C25.9,Large B-cell lymphoma arising in HHV8-associat...,372003004,Primary malignant neoplasm of pancreas
3,9560/0,C30.1,Neurilemoma of middle ear,985004,Neurilemoma
4,9560/0,C47.0,Neurilemoma of peripheral nerves and autonomic...,985004,Neurilemoma


In [ ]:
head1.merge(head2,how='right',on=)

In [311]:
#cleanest_df.to_csv("data/andrew_test.csv")